In [1]:
import texas_hold_em_utils.card as card
import texas_hold_em_utils.deck as deck
import texas_hold_em_utils.game as game
import texas_hold_em_utils.player as player
import pandas as pd

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import os
import sqlalchemy

conn = sqlalchemy.create_engine(os.getenv("SUPABASE_CONN_STRING")).connect()

In [115]:
1000 // player_count

142

In [168]:
player_count = 12

In [169]:
count = conn.execute(sqlalchemy.text(f"SELECT COUNT(*) FROM poker.random_games_for_{player_count}_players")).fetchone()[0]

In [170]:
offset = 0

data = pd.read_sql_query(sqlalchemy.text(f"SELECT * FROM poker.random_games_for_{player_count}_players LIMIT {count / (player_count - 1)}"), conn)

In [171]:
for player in range(player_count):
    data[f'player_{player}_card_1_rank'] = data[f'player_{player}_card_1'].apply(lambda x: card.Card().from_name(x).rank)
    data[f'player_{player}_card_1_suit'] = data[f'player_{player}_card_1'].apply(lambda x: card.Card().from_name(x).suit)
    data[f'player_{player}_card_2_rank'] = data[f'player_{player}_card_2'].apply(lambda x: card.Card().from_name(x).rank)
    data[f'player_{player}_card_2_suit'] = data[f'player_{player}_card_2'].apply(lambda x: card.Card().from_name(x).suit)

In [172]:
records = pd.DataFrame(columns=["card_1_rank", "card_2_rank", "suited", "wins", "counts"])

for card_1 in range(13):
    for card_2 in range(13):
        for suited in [True, False]:
            wins = 0.0
            counts = 0.0
            for player_num in range(player_count):
                relevant_data = data[data[f"player_{player_num}_card_1_rank"] == card_1]
                relevant_data = relevant_data[relevant_data[f"player_{player_num}_card_2_rank"] == card_2]
                relevant_data = relevant_data[(relevant_data[f"player_{player_num}_card_1_suit"] == relevant_data[f"player_{player_num}_card_2_suit"]) == suited]
                counts += relevant_data.shape[0]
                relevant_data = relevant_data[relevant_data["winners"].str.contains(f"player_{player_num}")]
                for comma_count in range(player_count):
                    part_wins = relevant_data[relevant_data["winners"].str.count(",") == comma_count]
                    wins += (part_wins.shape[0] / (comma_count + 1))
            records = pd.concat([records, pd.DataFrame([[card_1, card_2, suited, wins, counts]], columns=["card_1_rank", "card_2_rank", "suited", "wins", "counts"])])
            
                

C:\Users\amarl\AppData\Local\Temp\ipykernel_4092\4233784622.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  records = pd.concat([records, pd.DataFrame([[card_1, card_2, suited, wins, counts]], columns=["card_1_rank", "card_2_rank", "suited", "wins", "counts"])])


In [173]:
records = records[records["counts"] > 0]
records['win_rate'] = records['wins'] / records['counts']
records.sort_values(by="win_rate", ascending=False, inplace=True)
records

,card_1_rank,card_2_rank,suited,wins,counts,win_rate
0,12,12,False,147.000000,466.0,0.315451
0,11,11,False,115.000000,449.0,0.256125
0,8,9,True,36.000000,159.0,0.226415
0,10,10,False,95.500000,444.0,0.215090
0,11,6,True,30.000000,147.0,0.204082
...,...,...,...,...,...,...
0,0,1,False,16.833333,415.0,0.040562
0,0,5,False,17.166667,428.0,0.040109
0,2,0,False,18.000000,468.0,0.038462
0,4,0,False,18.583333,489.0,0.038003


In [174]:
records['rank'] = range(records.shape[0])
records['percentile'] = (records.shape[0] - records['rank']) / records.shape[0]
records

,card_1_rank,card_2_rank,suited,wins,counts,win_rate,rank,percentile
0,12,12,False,147.000000,466.0,0.315451,0,1.000000
0,11,11,False,115.000000,449.0,0.256125,1,0.996923
0,8,9,True,36.000000,159.0,0.226415,2,0.993846
0,10,10,False,95.500000,444.0,0.215090,3,0.990769
0,11,6,True,30.000000,147.0,0.204082,4,0.987692
...,...,...,...,...,...,...,...,...
0,0,1,False,16.833333,415.0,0.040562,320,0.015385
0,0,5,False,17.166667,428.0,0.040109,321,0.012308
0,2,0,False,18.000000,468.0,0.038462,322,0.009231
0,4,0,False,18.583333,489.0,0.038003,323,0.006154


In [175]:
records['player_count'] = player_count

In [176]:
from texas_hold_em_utils.sklansky import sklansky_rank, sklansky_playable_position

def find_sklansky(row):
    card_1 = card.Card().from_ints(row['card_1_rank'], 0)
    card_2 = card.Card().from_ints(row['card_2_rank'], 0 if row['suited'] else 1)
    return sklansky_rank(card_1, card_2)

records['sklansky'] = records.apply(find_sklansky, axis=1)
records['sklansky_position'] = records['sklansky'].apply(lambda x: sklansky_playable_position(x))
records

,card_1_rank,card_2_rank,suited,wins,counts,win_rate,rank,percentile,player_count,sklansky,sklansky_position
0,12,12,False,147.000000,466.0,0.315451,0,1.000000,12,1,"Early, Middle, Late"
0,11,11,False,115.000000,449.0,0.256125,1,0.996923,12,1,"Early, Middle, Late"
0,8,9,True,36.000000,159.0,0.226415,2,0.993846,12,3,"Early, Middle, Late"
0,10,10,False,95.500000,444.0,0.215090,3,0.990769,12,1,"Early, Middle, Late"
0,11,6,True,30.000000,147.0,0.204082,4,0.987692,12,7,Late if no bettors
...,...,...,...,...,...,...,...,...,...,...,...
0,0,1,False,16.833333,415.0,0.040562,320,0.015385,12,9,Not playable
0,0,5,False,17.166667,428.0,0.040109,321,0.012308,12,9,Not playable
0,2,0,False,18.000000,468.0,0.038462,322,0.009231,12,9,Not playable
0,4,0,False,18.583333,489.0,0.038003,323,0.006154,12,9,Not playable


In [177]:
avgs = records.groupby("sklansky")["win_rate"].mean()

cutoffs = []
for rank in range(8):
    cutoffs.append((avgs.iloc[rank] + avgs.iloc[rank + 1]) / 2)

def find_modified_sklansky(win_rate, cutoffs):
    for rank in range(8):
        if win_rate >= cutoffs[rank]:
            return rank + 1
    return 9

records['modified_sklansky'] = records['win_rate'].apply(find_modified_sklansky, args=(cutoffs,))
records

,card_1_rank,card_2_rank,suited,wins,counts,win_rate,rank,percentile,player_count,sklansky,sklansky_position,modified_sklansky
0,12,12,False,147.000000,466.0,0.315451,0,1.000000,12,1,"Early, Middle, Late",1
0,11,11,False,115.000000,449.0,0.256125,1,0.996923,12,1,"Early, Middle, Late",1
0,8,9,True,36.000000,159.0,0.226415,2,0.993846,12,3,"Early, Middle, Late",1
0,10,10,False,95.500000,444.0,0.215090,3,0.990769,12,1,"Early, Middle, Late",1
0,11,6,True,30.000000,147.0,0.204082,4,0.987692,12,7,Late if no bettors,1
...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,1,False,16.833333,415.0,0.040562,320,0.015385,12,9,Not playable,9
0,0,5,False,17.166667,428.0,0.040109,321,0.012308,12,9,Not playable,9
0,2,0,False,18.000000,468.0,0.038462,322,0.009231,12,9,Not playable,9
0,4,0,False,18.583333,489.0,0.038003,323,0.006154,12,9,Not playable,9


In [178]:
records['modified_sklansky_position'] = records['modified_sklansky'].apply(lambda x: sklansky_playable_position(x))

In [179]:
records.to_sql("win_rates", conn, if_exists="append", index=False, schema="poker")

325

In [180]:
conn.commit()

In [186]:
conn.commit()